<a href="https://colab.research.google.com/github/nithinbadicodes/tensorflow-food101/blob/master/tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow
* Exercises

## Introduction to Tensors

In [1]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.18.0


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
a = tf.Variable(7)
a.numpy()

np.int32(7)

### Creating random tensors

Random tensors are tensors of abitrary values

In [4]:
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

In [5]:
random_2 = tf.random.Generator.from_seed(107)
random_2 = random_2.normal(shape=(3,2))

In [6]:
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-1.5446119 , -0.9279166 ],
       [ 1.5999255 , -0.88942885],
       [-2.2454505 , -0.7396203 ]], dtype=float32)>

### Shuffle the order of elements in a tensor


In [7]:
# Shuffle a tensor
not_shuffled = tf.constant([[10,7],
                            [3,4],
                            [2,5]])
not_shuffled.ndim

2

In [8]:
# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

#### Its interactions with operation-level seeds is as follows:

* If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.
* If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.
* If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.
* If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [9]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [10]:
# Create a tensor of all ones
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and tensorFlow tensors is that tensors can be run on a GPU

In [11]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1,25,dtype=np.int32)
numpy_A
# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [12]:
A = tf.constant(numpy_A, shape=(2,3,4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors

* Shape
* Rank
* Dimension
* Size

In [13]:
rank_4_tensor = tf.zeros(shape=(2,3,4,5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [14]:
print("Datatype of every element: ",rank_4_tensor.dtype)
print("Number of dimensions (rank): ",rank_4_tensor.ndim)
print("Shape of the tensor: ",rank_4_tensor.shape)
print("Elements along the 0 axis: ",rank_4_tensor.shape[0])
print("Elements along the last axis: ",rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ",tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ",tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of the tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [15]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [16]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1,:1,:1]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [17]:
# Create a rank two tensor (2 dimensions)
rank_2_tensor = tf.constant([[10,7],
                             [3,4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [18]:
# Get the last item of each of our rank 2 tensor
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [19]:
tf.reshape(rank_2_tensor,shape=(4,1))

<tf.Tensor: shape=(4, 1), dtype=int32, numpy=
array([[10],
       [ 7],
       [ 3],
       [ 4]], dtype=int32)>

In [20]:
# Add in extra dimenision to our rank 2 tensor
rank_3_tensor = rank_2_tensor[tf.newaxis,...,tf.newaxis]

In [21]:
rank_3_tensor

<tf.Tensor: shape=(1, 2, 2, 1), dtype=int32, numpy=
array([[[[10],
         [ 7]],

        [[ 3],
         [ 4]]]], dtype=int32)>

In [22]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

### Manipulating tensors (tensor operations)

**Basic operations**
`+`,`-`,`*`,`/`

In [23]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10,7],[3,4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [24]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [25]:
# Subtraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [26]:
# Divide
tensor / 10

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1. , 0.7],
       [0.3, 0.4]])>

In [27]:
# We can use the tensorflow built-in function too
tf.multiply(tensor,10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

### Matrix multiplication

In machine learning, matrix multiplication is one of the most used tensor operations

In [28]:
tf.matmul(tensor,tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [29]:
# Creat a tensor (3,2) tensor

X = tf.constant([[1,2],
                 [3,4],
                 [5,6]])

In [30]:

# Create a tensor (2,3) tensor
Y = tf.constant([[1,2,3],
                 [4,5,6]])

In [31]:
X,Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [32]:
X @ Y

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 9, 12, 15],
       [19, 26, 33],
       [29, 40, 51]], dtype=int32)>

In [33]:
X

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int32)>

In [34]:
tf.reduce_mean(X)

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [35]:
import plotly.express as px

### Find positional maximum and minimum



In [39]:
# Create a new tensor
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [41]:
# Find the positional maximum
tf.argmax(F),F[42]

(<tf.Tensor: shape=(), dtype=int64, numpy=42>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>)

In [44]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [45]:
# Find the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [46]:
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463667869567871>